In [8]:
import itertools
from collections import defaultdict
import numpy as np

#in: button
#out: length of button
def length_of(button):
    return (button[1] - button[0] + 1)/2

WE = defaultdict(lambda: None)
WE[0] = 0 
WE[1] = 1

#outputs the Wedderburn number for n
def W(n):
    if n == 0:
        return 0
    if n == 1:
        return 1
    if WE[n] == None: 
        if n % 2 == 1:
            s = sum([W(i) * W(n-i) for i in range(1, (n+1)//2)])
            WE[n] = s
        else:
            s = sum([W(i) * W(n-i) for i in range(1, n//2)])
            s += (W(n//2) * (W(n/2) + 1)) / 2  #W(n/2) choose 2 + W(n/2)
            WE[n] = s
    return WE[n]
    
# input: pairs of integers (n,t)
# output: the root split of s'th binary tree (in lex order) on n leaves in the form ((n_0,s_0),(n_1,s_1))
def root_split_finder(pair):
    n=pair[0]
    t = pair[1]
    if t >= W(n):
        raise ValueError("there is no tree for t = ", t)
    if n == 1:
        return ((1,0),(0,0))
    elif n % 2 == 1:
        i = 1
        while W(n-i) * W(i) <= t: 
            t-=W(n-i) * W(i)
            i += 1
        (s_1,s_2) = divmod(int(t),int(W(i)))
        return ((n-i,s_1),(i,s_2))
    else:
        i = 1
        while i <= n/2 and W(n-i) * W(i) <= t: 
            t-=W(n-i) * W(i)
            i+=1      
        (s_1,s_2) = divmod(int(t),int(W(i)))
        if i != n/2:
            return ((n-i,s_1),(i,s_2))
        else:
            y = W(n//2)
            j = 0
            r = 0
            while y-j-1 < t:
                t -= (y-j)
                j += 1
                r += 1
            return ((n//2,t+r),(n//2,j))    


#input: a tanglegrams ((n,s_0),(n,s_1),p)
#output: the canonical tanglegram ((m,t_0),(m,t_1),r) in the isom. class of the given tanglegram
def button_list(T):
    ltree = T[0]
    rtree = T[1]
    sigma = T[2]
    
    n = ltree[0]
    
    left_buttonizers = set()
    right_buttonizers = set()

    left_height_dct = { i: 0 for i in range(1,n+1) }
    right_height_dct = { i: 0 for i in range(1,n+1) }
    
    # input: pair of integers (n,s) denoting the tree and pair of integers (a,b) denoting the leaf number interval
    # output: the root split of s'th binary tree (in lex order) on n leaves in the form ((n_0,s_0),(n_1,s_1))
    def root_split_finder_with_left_buttons(pair, leaf_interval):
        (ltree, rtree) = root_split_finder(pair)
        if ltree == rtree:
            left_buttonizers.add(leaf_interval)
        if ltree[0] > 1:
            
            left_bound = leaf_interval[0]
            right_bound = leaf_interval[0] + ltree[0] - 1
            
            for leaf in range(left_bound, right_bound + 1):
                left_height_dct[leaf]+=1
            
            root_split_finder_with_left_buttons(ltree,(left_bound, right_bound))
        
        if rtree[0] > 1:
            
            left_bound = leaf_interval[0] + ltree[0]
            right_bound = leaf_interval[1]
            
            for leaf in range(left_bound, right_bound + 1):
                left_height_dct[leaf]+=1
                
            root_split_finder_with_left_buttons(rtree,(left_bound, right_bound))

    # input: pair of integers (n,s) denoting the tree and pair of integers (a,b) denoting the leaf number interval
    # output: the root split of s'th binary tree (in lex order) on n leaves in the form ((n_0,s_0),(n_1,s_1))
    def root_split_finder_with_right_buttons(pair,leaf_interval):
        (ltree, rtree) = root_split_finder(pair)
        if ltree == rtree:
            right_buttonizers.add( leaf_interval )
        
        if ltree[0] > 1:
            
            left_bound = leaf_interval[0]
            right_bound = leaf_interval[0] + ltree[0] - 1
            
            for leaf in range(left_bound, right_bound + 1):
                right_height_dct[leaf]+=1
            
            root_split_finder_with_right_buttons(ltree,(left_bound, right_bound))
        
        if rtree[0] > 1:
            
            left_bound = leaf_interval[0] + ltree[0]
            right_bound = leaf_interval[1]
            
            for leaf in range(left_bound, right_bound + 1):
                right_height_dct[leaf]+=1
                
            root_split_finder_with_right_buttons(rtree,(left_bound, right_bound))
    
    root_split_finder_with_left_buttons(ltree,(1,n))
    root_split_finder_with_right_buttons(rtree,(1,n))
    

    return ( left_buttonizers, right_buttonizers )


def perm_mult(perm_1,perm_2):
    return perm_2 * perm_1

def button_to_perm(but,n):
    return permutation.from_cycles(n, [ [i,i+length_of(but)] for i in range(but[0],but[0] + length_of(but)) ])

#return True if and only if perm stabilizes the closed interval of integers [1,i]
def stabilizes(perm,i):
    perm_other_class = Permutation(perm)
    # print([i in perm.fixed_points() for i in range(1,i+1) ])
    return all([i in perm_other_class.fixed_points() for i in range(1,i+1) ] )

import sage.combinat.permutation as permutation

n = 4

perm_id = Permutations(n).identity()

tau = Permutation([ 2,3,4,1 ])
sigma = Permutation([ 1,3,2,4 ])

generators = set({tau, sigma})

# print(tau.to_cycles())
# print(sigma.to_cycles())

# print("sigma * tau", perm_mult(sigma,tau)  )
# print(" tau*sigma ", perm_mult(tau,sigma) )

T = ((4,1), (4,1), tau )

dct = defaultdict( lambda: set( { perm_id } ) )

def filter(alpha,n):
    i = max( [ i for i in range(n-1) if stabilizes(alpha,i) ] )

    new_representative = True
    
    for gamma in dct[i]:
        gamma_inverse_alpha = perm_mult( gamma.inverse(), alpha)
        if stabilizes(  perm_mult( gamma.inverse(), alpha) , i+1 ):
            new_representative = False
            if gamma_inverse_alpha != perm_id:
                filter(gamma_inverse_alpha,n)
    if new_representative and alpha != perm_id:
        dct[i].add(alpha)

    return None

pair = button_list(T)
left_buts = pair[0]
left_buts = pair[1]

L_processed = set({})

def complete_coset_reps(L,n):
    el = (L - L_processed).pop()
    filter(el,n)
    L_processed.add(el)
    new_ones = [ el for coset in dct.values() for el in coset ]

    for pair in itertools.product(new_ones,new_ones):
        L.add(perm_mult(pair[0],pair[1]))
    
    L_next = L - L_processed 

    # print("finished filtered", dct)
    # print("finished filtered", new_ones)
    # print("processed so far ", L_processed)
    # print("next",L_next)
    
    if len(L_next) != 0:
        complete_coset_reps(L_next,n)
    
    return None

complete_coset_reps(generators,n)

for el in sorted(dct.keys()):
    print(el,dct[el])

0 {[1, 2, 3, 4], [2, 3, 4, 1], [3, 2, 4, 1], [4, 2, 1, 3]}
1 {[1, 2, 3, 4], [1, 4, 2, 3], [1, 3, 2, 4]}
2 {[1, 2, 3, 4], [1, 2, 4, 3]}


In [ ]:
#input: a tanglegram as a triple (L,R,sigma) 
#output: the same tanglegram as a graph
def tanglegram_into_graph(T):
    return None

#input: graphs G_1, G_2
#output: True iff G_1 and G_2 are isomorphic
#polynomially tractable in the max degrees of G_1 and G_2
def degree_tractable_graph_isomorphism(G_1,G_2)
    return None